## MMDS Computational Advertising

In [86]:
class Advertiser:
    def __init__(self, name, bid, ctr1, ctr2, ctr3, budget):
        self.name = name
        self.bid = bid
        self.ctr1 = ctr1
        self.ctr2 = ctr2
        self.ctr3 = ctr3
        self.budget = budget
        self.clicks = 0
        
    def expected_yield(self, slot):
        return self.bid * self.get_ctr_by_slot(slot)
    
    def get_ctr_by_slot(self, slot):
        return getattr(self, 'ctr' + str(slot + 1))
    
    def total_impressions(self, slot):
        return round(self.budget / self.expected_yield(slot))
    
    def add_clicks(self, clicks):
        self.clicks += clicks
    
    def calc_clicks(self, impressions, slot):
        return round(impressions * self.get_ctr_by_slot(slot))
    
    def spend(self, clicks):
        cost = clicks * self.bid
        self.budget -= cost
        

In [87]:
def initialize_advertisers():
    advertisers = []
    advertisers.append(Advertiser('A', 0.1, 0.015, 0.01, 0.005, 1))
    advertisers.append(Advertiser('B', 0.09, 0.016, 0.012, 0.006, 2))
    advertisers.append(Advertiser('C', 0.08, 0.017, 0.014, 0.007, 3))
    advertisers.append(Advertiser('D', 0.07, 0.018, 0.015, 0.008, 4))
    advertisers.append(Advertiser('E', 0.06, 0.019, 0.016, 0.010, 5))
       
    return advertisers

In [88]:
def allocate_slots(advertisers):
    
    slots = []
    
    for i in range(3):
        candidate = advertisers[0]
        max_expected_yield = candidate.expected_yield(i)
        for advertiser in advertisers[1:]: 
            expected_yield = advertiser.expected_yield(i)
            if max_expected_yield < expected_yield:
                max_expected_yield = expected_yield
                candidate = advertiser
        
        candidate.slot = i
        slots.append(candidate)
        advertisers.remove(candidate)
                
    return slots

In [89]:
def process_phase(slots, remaining_clicks):
    
    impressions = [a.total_impressions(a.slot) for a in slots]
    min_impressions = min(impressions)
    
    clicks = [a.calc_clicks(min_impressions, a.slot) for a in slots]
    
    complete = False
    if sum(clicks) < remaining_clicks:
        for a in slots:
            a.add_clicks(clicks[a.slot])
            a.spend(clicks[a.slot])
    else:
        complete = True
        sum_ctr = sum([a.get_ctr_by_slot(a.slot) for a in slots])
        clicks = [round(remaining_clicks*(a.get_ctr_by_slot(a.slot)/sum_ctr)) for a in slots]
        for a in slots:
            a.add_clicks(clicks[a.slot])
        
    return complete, sum(clicks), slots    
    
    

In [90]:
advertisers = initialize_advertisers()
remaining_clicks = 101
total_clicks = {}
for i in range(20):
    slots = allocate_slots(advertisers)
    complete, clicks, slots = process_phase(slots, remaining_clicks)
    
    if complete:
        advertisers.extend(slots)
        for a in advertisers:
            total_clicks[a.name] = a.clicks
        print 'Complete'
        break
    else:
        remaining_clicks -= clicks
        remove_slot = slots[0].slot
        for a in slots:
            if a.budget < 0.05:
                total_clicks[a.name] = a.clicks
                remove_slot = a.slot
        del slots[remove_slot]
        advertisers.extend(slots)

print total_clicks

Complete
{'A': 10.0, 'C': 36.0, 'B': 22.0, 'E': 26.0, 'D': 7.0}
